In [1]:
import pandas as pd
from collections import Counter
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
trials = pd.read_excel('bcdrugsct.xlsx')
trials = trials[trials['Primary Drugs'].isna()==False]
cols = ['Trial ID','Mechanism of action', 'Drug class (therapeutic effect)', 'Drug class (chemical)', \
        'Indication', 'Organisations', 'Trial Design', 'Location', 'Phase of Trial', 'Subject Age',\
        'Planned Subject Number', 'Trial Centre Details', 'Lead Centre', 'Trial Initiation date', \
        'Trial End date', 'Trial Status', 'Trial History', 'Diseases treated', 'Primary Drugs']
# smaller_trials = trials[cols]
date_cols = ['Phase of Trial','Trial Initiation date', 'Trial End date', 'trial_start','trial_end']
if trials['Primary Drugs'].isna().sum() > 0:
    print('did not remove all na drug rows')

In [3]:
#not DRY as the answers to make multiargument apply functions involved more complicated things than I wanted.
def start_to_datetime(x):
    date_str = re.findall('[\d\s\w]*', x['Trial Initiation date'])[0].strip()
    if date_str == '':
        return False
    else:
        return pd.to_datetime(date_str)

def end_to_datetime(x):
    date_str = re.findall('[\d\s\w]*', x['Trial End date'])[0].strip()
    if date_str == '':
        return False
    else:
        return pd.to_datetime(date_str)

def phase_1(drug_name):
    '''
    Use a drug name and get out the probabilitiy that it passes multiple phases 
    and the times for it to do so.
    '''
    #get df with all rows related to drug
    drug_trials = trials[trials['Primary Drugs'].str.contains(drug_name,case=False)] 
    #drug_trials = trials[trials['Primary Drugs']==drug_name] 
    
    #see if the drug has phase I and II in data.
    unique_trial_values = drug_trials['Phase of Trial'].unique()
    pass_phase_1, pass_phase_2, pass_phase_3 = False, False, False
    pass_phase_one_two, pass_phase_two_three = False, False
    if ('Phase I' in unique_trial_values) & ('Phase II' in unique_trial_values):
        pass_phase_1 = True
    
    if ('Phase II' in unique_trial_values) & ('Phase III' in unique_trial_values):
        pass_phase_2 = True
        
    if ('Phase III' in unique_trial_values) & ('Phase IV' in unique_trial_values):
        pass_phase_3 = True
    
    if ('Phase I/II' in unique_trial_values) & ('Phase III' in unique_trial_values):
        pass_phase_one_two = True
    
    if ('Phase II/III' in unique_trial_values) & ('Phase IV' in unique_trial_values):
        pass_phase_two_three = True
    
    if pass_phase_1 == False:
        return [drug_name,False,False,False, False, False, False]
        
    #get rid of rows outside Phase I or II
    trial_phases = ['Phase I', 'Phase II']
    cleaned_drug_trials = drug_trials[drug_trials['Phase of Trial'].isin(trial_phases)] #only rows w/ clean trials        

    #turn categorial variables into ordinals
    phase_dummies = pd.get_dummies(cleaned_drug_trials['Phase of Trial'])
    df = cleaned_drug_trials.join(phase_dummies)
    
    #convert times to time objects
    df['trial_start'] = df.apply(start_to_datetime, axis=1)
    df['trial_end'] = df.apply(end_to_datetime, axis=1)
     
    #if len(df[df['trial_start']==False])>0:
    #    return f'{drug_name} error'
    
    #finds only the dates that aren't false for each phase.
    phase_1_start = min(list(df[(df['trial_start']!=False) & (df['Phase of Trial']=='Phase I')]['trial_start'].values))
    phase_1_first_end = min(list(df[(df['trial_start']!=False) & (df['Phase of Trial']=='Phase II')]['trial_start'].values))
    phase_1_last_end = max(list(df[(df['trial_start']!=False) & (df['Phase of Trial']=='Phase I')]['trial_start'].values))
    
    #return (f'min is {t_phase_1_min} and max is {t_phase_1_max}')
    return [drug_name, pass_phase_1,pass_phase_2, pass_phase_3, pass_phase_one_two, pass_phase_two_three, phase_1_start, phase_1_first_end, phase_1_last_end]



In [4]:
unique_drug_list =list(trials['Primary Drugs'].unique())
unique_drugs = {drug for row in unique_drug_list for drug in row.split(', ')}

drug_counts = trials['Primary Drugs'].value_counts()
drugs_10_trials = drug_counts[drug_counts>1].index
all_drugs = drug_counts.index

In [5]:
len(unique_drugs)

1001

In [6]:
c = Counter(np.array([drug for row in unique_drug_list for drug in row.split(', ')]))

In [7]:
res, except_ids = [],[]
for idx,i in enumerate(unique_drugs):
    try:
        res.append(phase_1(i))
    except:
        except_ids.append(idx)

print (f'got {len(res)} results and had {len(except_ids)} exceptions')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


got 983 results and had 18 exceptions


In [8]:
drugs_df = pd.DataFrame(res, columns=['drug','phase_1_success','phase_2_success','phase_3_success','phase_1_2_success','phase_2_3_success','phase_1_start', 'phase_1_first_end', 'phase_1_last_end'])
drugs_df.shape

(983, 9)

In [9]:
ans = [drugs_df[drugs_df['phase_1_success']==True].shape[0]/drugs_df.shape[0], 
drugs_df[drugs_df['phase_2_success']==True].shape[0]/drugs_df.shape[0], 
drugs_df[drugs_df['phase_3_success']==True].shape[0]/drugs_df.shape[0]]
[round(100*i,2) for i in ans]

[21.36, 6.51, 3.46]